In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import pairwise_distances
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine

# Get the current working directory
current_dir = os.getcwd()

# Append the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


# ignore warrnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
# import Postgres connection from DB_connection folder
from DB_connection.connection import PostgresConnection
from scripts.satisfaction_analyzer import SatisfactionAnalyer
from scripts.engagement_analyser import EngagementAnalyzer
from scripts.experience_analyzer import ExperienceAnalyzer

In [7]:
# Create an instance of the PostgresConnection class and connect to the database
db=PostgresConnection()
db.connect()

Connected to PostgreSQL database


In [8]:
# query to retrive all the data from xdr_data table
query='select * from xdr_data'
db.execute_query(query)

Query executed successfully


In [9]:
# Assign the data to a dataframe named 'df'
df=db.fetch_data(query)
df.head()

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,15854611.0,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,20247395.0,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,19725661.0,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,21388122.0,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,15259380.0,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0


### User Engagement 

In [10]:
satisfaction_analyser=SatisfactionAnalyer(df)

In [12]:
user_engagement_df = satisfaction_analyser.user_engagement(df)

  File "e:\Kiffya_10_acc\Week 2\TellCo-Telecom-Analysis\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Getahun\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Getahun\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\Getahun\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [13]:
user_engagement_df

,MSISDN/Number,Session Frequency,Total Duration,Total UL (Bytes),Total DL (Bytes),Total Traffic (Bytes),Engagement Cluster
0,3.360100e+10,0.055556,0.095931,36053108.0,8.426375e+08,0.095931,2
1,3.360100e+10,0.055556,0.014026,36104459.0,1.207552e+08,0.014026,0
2,3.360100e+10,0.055556,0.063851,39306820.0,5.566597e+08,0.063851,0
3,3.360101e+10,0.055556,0.044148,20327526.0,4.019932e+08,0.044148,0
4,3.360101e+10,0.111111,0.161598,94280527.0,1.363130e+09,0.161598,2
...,...,...,...,...,...,...,...
106851,3.379000e+10,0.055556,0.077317,26716429.0,6.879252e+08,0.077317,2
106852,3.379000e+10,0.055556,0.050727,35732243.0,4.445751e+08,0.050727,0
106853,3.197021e+12,0.055556,0.022566,37295915.0,1.948281e+08,0.022566,0
106854,3.370000e+14,0.055556,0.063887,56652839.0,5.396350e+08,0.063887,0


In [14]:
# Selecting only the relevant columns for normalization
metrics = ['Session Frequency', 'Total Duration', 'Total Traffic (Bytes)']

least_engaged_cluster=satisfaction_analyser.get_least_engaged_cluster(user_engagement_df,'Engagement Cluster',metrics)


                    Session Frequency  Total Duration  Total Traffic (Bytes)
Engagement Cluster                                                          
0                            0.057815        0.037848               0.037848
1                            0.194881        0.218017               0.218017
2                            0.081787        0.097088               0.097088
The least engaged cluster is: 0


#### User Experiences

In [15]:
user_experience_df=pd.read_csv("user_experience_data.csv")
user_experience_df.drop(columns=['Handset Type']).head()

FileNotFoundError: [Errno 2] No such file or directory: 'user_experience_data.csv'

In [ ]:
metrics=['Avg TCP DL Retransmission', 'Avg RTT DL', 'Avg Throughput DL']
worst_experience_cluster = satisfaction_analyser.get_worst_experience_cluster(user_experience_df,'Experience Cluster',metrics)

: 

In [ ]:
# We have have the user engagement data (user_engagement_df) and experience data (user_experience_df) from Task-2 and Task-3

# Step 1: Calculate the centroids of the least engaged cluster and worst experience cluster
least_engaged_centroid = user_engagement_df[user_engagement_df['Engagement Cluster'] == least_engaged_cluster].mean().values
# Get the centroid for the worst experience cluster
worst_experience_centroid = user_experience_df[user_experience_df['Experience Cluster'] == worst_experience_cluster][metrics].mean().values

: 

In [ ]:
worst_experience_centroid

: 

In [ ]:
user_engagement_df, user_experience_df = satisfaction_analyser.calculate_scores(user_engagement_df,user_experience_df)

: 

In [ ]:
user_engagement_df

: 

In [ ]:
user_experience_df

: 

In [ ]:
# Calculate satisfaction score and report top 10 satisfied customers
merged_df, top_10_satisfied = satisfaction_analyser.calculate_satisfaction(user_engagement_df,user_experience_df)

: 

In [ ]:
# Merged dataframe with the same index
merged_df

: 

In [ ]:
# display the top 10 rows of the DataFrame
top_10_satisfied
  

: 

In [ ]:
# Plot the top 10 satisfied customers
satisfaction_analyser.plot_top_10_satisfied(top_10_satisfied)

: 

In [ ]:
model, coefficients, intercept = satisfaction_analyser.build_regression_model(merged_df)

: 

In [ ]:
# K-means clustering on the engagement & experience scores
merged_df, kmeans_model = satisfaction_analyser.perform_kmeans_clustering(merged_df)

: 

In [ ]:
#  Display merged data after clustering
merged_df

: 

In [ ]:
# Aggregate the average satisfaction & experience score per cluster
cluster_aggregation = satisfaction_analyser.aggregate_scores_per_cluster(merged_df)
print(cluster_aggregation)

: 

In [ ]:
# Task 4.4 - K-means clustering on the engagement & experience scores
X_cluster = merged_df[['Engagement Score', 'Experience Score']]
scaler = StandardScaler()
X_cluster_scaled = scaler.fit_transform(X_cluster)
kmeans = KMeans(n_clusters=2, random_state=42)
merged_df['Cluster'] = kmeans.fit_predict(X_cluster_scaled)

# Task 4.5 - Aggregate the average satisfaction & experience score per cluster
cluster_aggregation = merged_df.groupby('Cluster').agg({
    'Satisfaction Score': 'mean',
    'Experience Score': 'mean'
})
cluster_aggregation

: 

#### Export user satsfaction score into postgreSQL data base

In [ ]:
db_config = {
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',  # or your MySQL server address
    'database': 'teleco'
}
user_score_data = merged_df[['MSISDN/Number', 'Engagement Score', 'Experience Score', 'Satisfaction Score']]
satisfaction_analyser.export_to_mysql(user_score_data,table_name='user_score',db_config=db_config)

: 

In [ ]:
user_score_data

: 